In [1]:
import cplex

# Boggle Board Generator

We have 16 squares, indexed by $j=0,...,15$.


| The grid: |     |     |     |     |
| ---       | --- | --- | ----| --- |
|           |  0  |  1  |  2  |  3  |
|           |  4  |  5  |  6  |  7  |
|           |  8  |  9  |  10 |  11 | 
|           | 12  |  13 |  14 |  15 |


We have the set of words that must appear $W$.  For some word $w \in W$, the length of word $w$ is $m_w$.  Let $i$ be the index of the $i^{th}$ letter of the word, where $i=0,...,m_w-1$.  Notationally we may refer to the letter as $w_i$.  The words are composed of the unique set of letters $A$, where $|A|=n$.  The maximum word length is 16.

Define the "decision variables" $x$ as:

"x_{wij} = \\begin{cases} 1 & \\mbox{ if word } w { letter index } i \\mbox{ is assigned to square } j, \\\\\n",
    "                       0 & \\mbox{ otherwise }\n",
    "         \\end{cases}\n",
    "\\end{align}\n",

                   
It's worth noting here that we have $16\timxs16|W|$ variables.  That means we have $2^{256|W|}$ possible solutions.  Not all solutions will be valid.  If a solution satisfies the following constraints, then it is valid.

Only one letter is assigned per square:
\begin{align}
\sum_j x_{wij} &= 1 && \forall w, \forall i
\end{align}


Letter $w_i$ appears in at least one square (for each word $w$, each letter $w_i$).
\begin{align}
\sum_j x_{wij} &\ge 1 && \forall w, \forall i
\end{align}


Let $\Omega_i$ be the set of neighbors of square $i$ with respect to the grid indexes above.  For example:
\begin{align}
\Omega_0 &= \{ 1, 4, 5 \} \\
\Omega_5 &= \{ 0, 1, 2, 4, 6, 8, 9, 10 \}\\
\mbox{etc.}
\end{align}

For each word $w \in W$, consider the sequence of letters $w_0w_1...w_{m-1}$
\begin{align}
\sum_{\ell \in \Omega_i} x_{\ell w_{k+1}} &\ge x_{iw_k} && \forall i=0,...,15, & \forall k=0,...,m-1 , & \forall & w \in W
\end{align}

Each word $w \in W$ must start in some where.
\begin{align}
\sum_j x_{w0j} &= 1 \forall w \in W 


In [2]:
word_list = ["square","gold","blue","toast"]

filler_string = "X"
alphabet = [filler_string] #always have some filler letter(s)

for w in word_list:
    for wk in range(len(w)):
        if w[wk] not in alphabet:
            alphabet.append(w[wk])

print(alphabet)

['X', 's', 'q', 'u', 'a', 'r', 'e', 'b', 't', 'h', 'o', 'l']


In [3]:
Omega = [] #init
Omega.append([1,4,5]) # 0
Omega.append([0,2,4,5,6]) # 1
Omega.append([1,3,5,6,7]) # 2
Omega.append([2,6,7]) # 3
Omega.append([0,1,5,8,9]) # 4
Omega.append([0,1,2,4,6,8,9,10]) # 5
Omega.append([1,2,3,5,7,9,10,11]) # 6
Omega.append([2,3,6,10,11]) # 7
Omega.append([4,5,9,12,13]) # 8
Omega.append([4,5,6,8,10,12,13,14]) # 9
Omega.append([5,6,7,9,11,13,14,15]) # 10
Omega.append([6,7,10,14,15]) # 11
Omega.append([8,9,13]) # 12
Omega.append([8,9,10,12,14]) # 13
Omega.append([9,10,11,13,15]) # 14
Omega.append([10,11,14]) # 15

for i in range(len(Omega)):
    print("Omega",i,":",Omega[i])

Omega 0 : [1, 4, 5]
Omega 1 : [0, 2, 4, 5, 6]
Omega 2 : [1, 3, 5, 6, 7]
Omega 3 : [2, 6, 7]
Omega 4 : [0, 1, 5, 8, 9]
Omega 5 : [0, 1, 2, 4, 6, 8, 9, 10]
Omega 6 : [1, 2, 3, 5, 7, 9, 10, 11]
Omega 7 : [2, 3, 6, 10, 11]
Omega 8 : [4, 5, 9, 12, 13]
Omega 9 : [4, 5, 6, 8, 10, 12, 13, 14]
Omega 10 : [5, 6, 7, 9, 11, 13, 14, 15]
Omega 11 : [6, 7, 10, 14, 15]
Omega 12 : [8, 9, 13]
Omega 13 : [8, 9, 10, 12, 14]
Omega 14 : [9, 10, 11, 13, 15]
Omega 15 : [10, 11, 14]


In [4]:
c = cplex.Cplex() #init model

c.set_problem_name("boggle_generator")

n = 16
m = len(alphabet)



# set objective to prefer the FILLER word.  Just for sanity.  Not technically required.
c.objective.set_sense(c.objective.sense.maximize)
x_var_names = []
obj_coeffs = []
for i in range(n):
    for j in alphabet:
        x_var_names.append("x_" + str(i) + "_" + j)
        if j == filler_string:
            obj_coeffs.append(1.0) #prefer filler
        else:
            obj_coeffs.append(0.0)
c.variables.add(names = x_var_names,
               types=[c.variables.type.binary]*(n*m),
               obj=obj_coeffs)




# constraint: only one letter per square
for i in range(n):
    var_list = []
    coeff_list = []
    for j in alphabet:
        var_list.append("x_" + str(i) + "_" + j)
        coeff_list.append(1.0)
    c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list, val=coeff_list)],
                            senses=["E"],
                            rhs=[1.0],
                            names=["only_one_letter_in_square_" + str(i)])

# constraint: letter j appears in at least one square
for j in alphabet:
    var_list = []
    coeff_list = []
    for i in range(n):
        var_list.append("x_" + str(i) + "_" + j)
        coeff_list.append(1.0)
    c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list, val=coeff_list)],
                             senses=["G"],
                             rhs=[1.0],
                             names=["letter_" + j + "_must_appear_in_some_square"])


# constraint: consecutive letters need to appear within the omega neighborhood
for w in word_list:
    for wk in range(len(w)-1):
        j = w[wk]
        next_j = w[wk+1]
        for i in range(n):
            var_list = []
            coeff_list = []
            var_list.append("x_" + str(i) + "_" + j)
            coeff_list.append(-1.0)
            for ell in Omega[i]:
                var_list.append("x_" + str(ell) + "_" + next_j)
                coeff_list.append(1.0)
            c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list,val=coeff_list)],
                                    senses=["G"],
                                    rhs=[0.0],
                                    names=[w + "_" + j + "_" + next_j + "_square_" + str(i)])
            

    
# write the model to a file
c.write(c.get_problem_name() + ".lpt")

print(c.get_stats())

Problem name         : boggle_generator
Objective sense      : Maximize
Variables            :     192  [Binary: 192]
Objective nonzeros   :      16
Linear constraints   :     204  [Greater: 188,  Equal: 16]
  Nonzeros           :    1484
  RHS nonzeros       :      28

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   : 1.000000         Max   : 1.000000       
Linear constraints   :
  Nonzeros           : Min   : 1.000000         Max   : 1.000000       
  RHS nonzeros       : Min   : 1.000000         Max   : 1.000000       



In [5]:
#Optional: read in problem from local file
#c.read("xxxxxx") 

c.solve()

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 204 rows, 192 columns, and 1484 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.72 ticks)
Found incumbent of value 1.000000 after 0.02 sec. (2.35 ticks)
Probing time = 0.00 sec. (0.43 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 204 rows, 192 columns, and 1484 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.16 ticks)
Probing time = 0.00 sec. (0.43 ticks)
Clique table members: 16.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (1.80 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt  

In [6]:
var_names = c.variables.get_names()
solution_values = c.solution.get_values()

for v in range(len(var_names)):
    if abs(solution_values[v]) > 1e-9: #only print nonzero values
        print(var_names[v],":",solution_values[v])

x_0_X : 1.0
x_1_b : 1.0
x_2_a : 1.0
x_3_q : 1.0
x_4_X : 1.0
x_5_r : 1.0
x_6_u : 1.0
x_7_s : 1.0
x_8_t : 1.0
x_9_t : 1.0
x_10_e : 1.0
x_11_X : 1.0
x_12_X : 1.0
x_13_l : 1.0
x_14_o : 1.0
x_15_h : 1.0


In [7]:
grid = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


for v in range(len(var_names)):
    if abs(solution_values[v]) > 1e-9:
        i = var_names[v].split("_")[1]
        j = var_names[v].split("_")[2]
        
        row = int(i) // 4
        col = int(i) % 4
        grid[row][col] = j
        

for row in grid:
    print(row)

['X', 'b', 'a', 'q']
['X', 'r', 'u', 's']
['t', 't', 'e', 'X']
['X', 'l', 'o', 'h']


This was a first draft.  There are some details that need to be worked out.

1. Make filler characters optional if the board is filled completely with valid words.
2. I think I need to declare another variable ($y$) as the "starting square" for each word.  I think right now the $\Omega$ constraints are actually too tight.